# MNIST 손글씨 - CNN 2
 - 2 Conv2d layers
 - 1 Maxpooling 2d layer
 - 1 Fully Connected Network layer
 - 1 Output layer


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed=2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import mnist
(X_train,y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# 0~1 사이의 값으로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# 3차원 데이터를 4차원으로 (데이터는 2차원을 3차원으로)
X_train = X_train.reshape(-1,28,28,1)    # 28*28=784이므로
X_test = X_test.reshape(-1,28,28,1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
# One hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [8]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=(28,28,1), activation='relu'),  # 마스크개수, 커널사이즈, 인풋사이즈, 함수
    Conv2D(64, (3,3), input_shape=(28,28,1), activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),   # 3차원을 1차원으로 변환
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4719104   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 4,743,050
Trainable params: 4,743,050
Non-trainable params: 0
______________________________________________

In [9]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [11]:
# 모델 최적화를 위한 설정
modelpath = 'best_mnist_cnn_2.h5'
checkpoint = ModelCheckpoint(modelpath, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(patience=10)

In [12]:
# 모델 학습
history = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=50, batch_size=200, verbose=0,
    callbacks=[checkpoint,early_stopping]
)


Epoch 00001: val_loss improved from inf to 0.05873, saving model to best_mnist_cnn_2.h5

Epoch 00002: val_loss improved from 0.05873 to 0.04669, saving model to best_mnist_cnn_2.h5

Epoch 00003: val_loss improved from 0.04669 to 0.04305, saving model to best_mnist_cnn_2.h5

Epoch 00004: val_loss improved from 0.04305 to 0.04104, saving model to best_mnist_cnn_2.h5

Epoch 00005: val_loss did not improve from 0.04104

Epoch 00006: val_loss did not improve from 0.04104

Epoch 00007: val_loss did not improve from 0.04104

Epoch 00008: val_loss did not improve from 0.04104

Epoch 00009: val_loss did not improve from 0.04104

Epoch 00010: val_loss did not improve from 0.04104

Epoch 00011: val_loss did not improve from 0.04104

Epoch 00012: val_loss did not improve from 0.04104

Epoch 00013: val_loss did not improve from 0.04104

Epoch 00014: val_loss did not improve from 0.04104


In [13]:
from tensorflow.keras.models import load_model
best_model = load_model(modelpath)
best_model.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0401 - accuracy: 0.9871


[0.040138352662324905, 0.9871000051498413]